# Route Simulation

## Route Network

In [114]:
from typing import List, Dict, Set

class Edge(object):
    def __init__(self, length:float, next_stop:Stop):
        self._length = length # Km
        self._next_stop = next_stop
        
    @property
    def length(self) -> int:
        return self._length
    
    @property
    def next_stop(self) -> Stop:
        return self._next_stop
    
    def __str__(self) -> str:
        return f"{self.length} Km to {self.next_stop}"
    
    def __repr__(self) -> str:
        return f"{self.length} Km to {self.next_stop}"

class Stop(object):
    def __init__(self, name:str, edges:Dict[str,Edge]=None):
        self._name = name
        self.edges = edges
        
    @property
    def name(self) -> str:
        return self._name
    
    def __str__(self) -> str:
        return f"{self.name}"
    
    def __repr__(self) -> str:
        return f"{self.name}"
    
class Route(object):
    def __init__(self, name:str):
        self._name = name
        self.stops = dict()
        self.buses = set()
        
    @property
    def name(self) -> str:
        return self._name
    
    def get_other_direction(self, cur_direction) -> str:
        for direction in stops:
            if direction != cur_direction:
                return direction
        return "No other direction"
    
    def add_stop(self, stop:Stop, direction:str):
        if direction not in self.stops:
            self.stops[direction] = list()
        self.stops[direction].append(stop)
        
    def add_bus(self, bus:Bus):
        self.buses.add(bus)
    
    def __str__(self) -> str:
        return f"{self.stops}"
    
    def __repr__(self) -> str:
        return f"{self.stops}"
            
class StopNetwork(object):
    def __init__(self):
        self.stops = dict() # Dictionary of Stop name -> Stop object
        self.routes = dict() # Dictionary of Route name -> Route object
        
    def add_edge(self, origin_name:str, dest_name:str, route_name:str, route_direction:str, length:float):
        # Populate stops dictionary
        if origin_name not in self.stops:
            self.stops[origin_name] = Stop(origin_name, {})
        if dest_name not in self.stops:
            self.stops[dest_name] = Stop(dest_name, {})
        self.stops[origin_name].edges[route_direction] = Edge(length, self.stops[dest_name])
        
        # Populate routes dictionary
        if route_name not in self.routes:
            self.routes[route_name] = Route(route_name)
        self.routes[route_name].add_stop(self.stops[origin_name], route_direction)
        
    def add_bus(self, route_name:str, route_direction:str, cur_stop_name:str, bus_id:int, speed:float):
        new_bus = Bus(bus_id, speed, self.stops[cur_stop_name], self.routes[route_name], route_direction)
        self.routes[route_name].add_bus(new_bus)
        
    def move_all_buses(self, timestep:float):
        for route in self.routes.values():
            for bus in route.buses:
                print(bus)
                bus.move(timestep)

## Bus model

In [115]:
class Bus(object):
    def __init__(self, bus_id:int, speed:float, cur_stop:Stop, route:Route, route_direction:str):
        self._id = bus_id
        self.speed = speed # Km/h
        self.route = route
        self.cur_stop = cur_stop
        self.route_direction = route_direction
        self.distance_to_next_stop = cur_stop.edges[route_direction].length
    
    def move(self, timestep:float): # timestep in seconds
        self.distance_to_next_stop -= timestep * self.speed / 3600
        if self.distance_to_next_stop <= 0: # arrived at next stop
            self.cur_stop = self.cur_stop.edges[self.route_direction].next_stop
            if self.route_direction not in self.cur_stop.edges: # Got to the end of direction
                self.route_direction = self.route.get_other_direction(self.route_direction)
            self.distance_to_next_stop = self.cur_stop.edges[self.route_direction].length
            
    @property
    def id(self) -> int:
        return self._id
    
    def __str__(self) -> str:
        return f"{self.id} {self.distance_to_next_stop} Km from {self.cur_stop}"
    
    def __repr__(self) -> str:
        return f"{self.id} {self.distance_to_next_stop} Km from {self.cur_stop}"

## Initialize stop network

In [116]:
stop_network = StopNetwork()

## M14D-SBS

In [117]:
direction = "M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST"
stop_names = [
    "DELANCEY ST/COLUMBIA ST",
    "COLUMBIA ST/RIVINGTON ST",
    "AV D/E HOUSTON ST",
    "AV D/E 5 ST",
    "E 10 ST/AV D",
    "AV C/E 11 ST",
    "E 14 ST/AV C",
    "E 14 ST/AV B",
    "E 14 ST/AV A",
    "E 14 ST/1 AV",
    "E 14 ST/2 AV",
    "E 14 ST/3 AV",
    "E 14 ST/4 AV",
    "E 14 ST/UNION SQ W",
    "W 14 ST/5 AV",
    "W 14 ST/6 AV",
    "W 14 ST/7 AV",
    "W 14 ST/8 AV",
    "W 14 ST/9 AV",
    "W 14 St/10 AV",
    "11 AV/W 15 ST",
    "11 AV / W 17 ST"
]
distances = [
    0.167,
    0.280,
    0.255,
    0.387,
    0.244,
    0.318,
    0.175,
    0.226,
    0.550,
    0.339,
    0.199,
    0.178,
    0.265,
    0.195,
    0.309,
    0.259,
    0.278,
    0.286,
    0.140,
    0.268,
    0.126
]
for i in range(1, len(stop_names)):
    origin_name = stop_names[i - 1]
    dest_name = stop_names[i]
    length = distances[i - 1]
    stop_network.add_edge(origin_name, dest_name, "M14D-SBS", direction, length)

In [118]:
for stop_name, stop in stop_network.stops.items():
    print(stop, stop.edges)

DELANCEY ST/COLUMBIA ST {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.167 Km to COLUMBIA ST/RIVINGTON ST}
COLUMBIA ST/RIVINGTON ST {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.28 Km to AV D/E HOUSTON ST}
AV D/E HOUSTON ST {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.255 Km to AV D/E 5 ST}
AV D/E 5 ST {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.387 Km to E 10 ST/AV D}
E 10 ST/AV D {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.244 Km to AV C/E 11 ST}
AV C/E 11 ST {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.318 Km to E 14 ST/AV C}
E 14 ST/AV C {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.175 Km to E 14 ST/AV B}
E 14 ST/AV B {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.226 Km to E 14 ST/AV A}
E 14 ST/AV A {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.55 Km to E 14 ST/1 AV}
E 14 ST/1 AV {'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': 0.339 Km to E 14 ST/2 AV}
E 14 ST/2

In [119]:
for route_name, route in stop_network.routes.items():
    print(route)

{'M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST': [DELANCEY ST/COLUMBIA ST, COLUMBIA ST/RIVINGTON ST, AV D/E HOUSTON ST, AV D/E 5 ST, E 10 ST/AV D, AV C/E 11 ST, E 14 ST/AV C, E 14 ST/AV B, E 14 ST/AV A, E 14 ST/1 AV, E 14 ST/2 AV, E 14 ST/3 AV, E 14 ST/4 AV, E 14 ST/UNION SQ W, W 14 ST/5 AV, W 14 ST/6 AV, W 14 ST/7 AV, W 14 ST/8 AV, W 14 ST/9 AV, W 14 St/10 AV, 11 AV/W 15 ST]}


In [120]:
stop_network.add_bus("M14D-SBS", direction, "DELANCEY ST/COLUMBIA ST", 4950, 10.33)

In [121]:
stop_network.move_all_buses(1)

4950 0.167 Km from DELANCEY ST/COLUMBIA ST


In [137]:
stop_network.move_all_buses(40)

4950 0.318 Km from AV C/E 11 ST
